# Assignment: Add Streaming Responses to LangGraph Nodes


---

### Objective:
This assignment challenges you to **implement streaming responses within a LangGraph workflow**. Traditionally, LLM responses are received as a single, complete block. However, for a better user experience, especially with longer generations, streaming allows you to display text as it's generated. You will build a simple content generation and summarization workflow, demonstrating how to stream output from the LLM nodes and integrate it into the overall LangGraph execution.

---

### Instructions:
1.  **LLM Access**: You'll need access to an LLM API that supports streaming. For this assignment, we'll primarily use **Google's Gemini Pro model** via the `langchain-google-genai` integration.
2.  **Environment Setup**: Install the necessary Python libraries: `pip install langchain-google-genai langgraph langchain_core`.
3.  **API Key**: Securely handle your API key. It's best practice to load it from an environment variable.
4.  **Jupyter Notebook**: All your code, outputs, observations, and analysis must be documented in this Jupyter Notebook.
5.  **Workflow Scenario**: You will create a two-node workflow:
    * **Content Generation Node**: Generates a short article.
    * **Summarization Node**: Summarizes the generated article.
6.  **Streaming Requirement**: Both the `Content Generation Node` and `Summarization Node` must demonstrate streaming behavior when invoking the LLM.
7.  **Analysis**: Discuss the implementation of streaming and its benefits.

---

## Part 1: Setup and Workflow State Definition
Configure your LLM and define the state your LangGraph workflow will manage.

### Task 1.1: API Configuration and LLM Initialization
Set up your Google Generative AI API key and initialize the `ChatGoogleGenerativeAI` model. Ensure it's configured for streaming.

In [ ]:
import os
from typing import TypedDict, Iterator

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import BaseMessage

from langgraph.graph import StateGraph, END

# --- YOUR API KEY HERE ---
# It's highly recommended to load your API key from an environment variable for security.
os.environ["GOOGLE_API_KEY"] = "YOUR_API_KEY_HERE" # Replace with your actual API key

# Initialize the LLM (Gemini Pro) - Streaming is usually enabled by default for supported models
llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.7, streaming=True)

print("LLM initialized with Gemini Pro and streaming enabled!")

### Task 1.2: Define Workflow State
Define a `TypedDict` to represent the state of your LangGraph workflow. This state will hold the generated content and its summary.

* `topic`: The subject for content generation.
* `generated_article`: The full generated article text.
* `summary`: The summary of the article.

In [ ]:
class ContentWorkflowState(TypedDict):
    topic: str
    generated_article: str
    summary: str

print("ContentWorkflowState defined!")

---

## Part 2: Define Nodes with Streaming Implementation
Create the nodes, ensuring they utilize the LLM's streaming capabilities and print the streamed chunks as they arrive.

### Task 2.1: `generate_article_node` with Streaming
This node will generate a short article based on the `topic`. It **must stream** the content and print each chunk to the console as it's received.

* **Prompt Design**: Create a prompt for article generation.
* **Streaming Logic**: Use `llm.stream()` or chain's `stream()` method. Iterate over the streamed chunks and print them. Accumulate the full content for the state.
* **Output**: Update `generated_article` in the state.

In [ ]:
generation_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Generate a short, informative article (around 200-300 words) on the given topic."),
    ("user", "Topic: {topic}")
])

generation_chain = generation_prompt | llm

def generate_article_node(state: ContentWorkflowState) -> ContentWorkflowState:
    print("\n--- Node: Generating Article (Streaming) ---")
    topic = state["topic"]
    full_article = ""

    print(f"Generating article on '{topic}':")
    for chunk in generation_chain.stream({"topic": topic}):
        if isinstance(chunk, BaseMessage) and chunk.content:
            print(chunk.content, end="", flush=True) # Print each chunk
            full_article += chunk.content
        elif isinstance(chunk, str):
            print(chunk, end="", flush=True)
            full_article += chunk

    print("\n--- Article Generation Complete ---")
    return {"generated_article": full_article}

print("generate_article_node defined!")

### Task 2.2: `summarize_article_node` with Streaming
This node will take the `generated_article` and produce a concise summary. It also **must stream** its output.

* **Prompt Design**: Create a prompt for summarization.
* **Streaming Logic**: Similar to the previous node, iterate and print chunks.
* **Output**: Update `summary` in the state.

In [ ]:
summarization_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Summarize the following article concisely, focusing on key points and main ideas."),
    ("user", "Article: {article_text}")
])

summarization_chain = summarization_prompt | llm

def summarize_article_node(state: ContentWorkflowState) -> ContentWorkflowState:
    print("\n--- Node: Summarizing Article (Streaming) ---")
    article_text = state["generated_article"]
    full_summary = ""

    print("Generating summary:")
    for chunk in summarization_chain.stream({"article_text": article_text}):
        if isinstance(chunk, BaseMessage) and chunk.content:
            print(chunk.content, end="", flush=True) # Print each chunk
            full_summary += chunk.content
        elif isinstance(chunk, str):
            print(chunk, end="", flush=True)
            full_summary += chunk

    print("\n--- Summary Generation Complete ---")
    return {"summary": full_summary}

print("summarize_article_node defined!")

---

## Part 3: Construct and Execute the LangGraph Workflow
Combine your streaming nodes into a `StateGraph` and execute it to observe the streaming behavior.

### Task 3.1: Define Edges and Compile the Graph
Set up the sequential flow for your content generation and summarization workflow.

* **Graph Flow**:
    1.  Start at `generate_article_node`.
    2.  From `generate_article_node`, go to `summarize_article_node`.
    3.  From `summarize_article_node`, `END` the workflow.

In [ ]:
# Define the graph
workflow = StateGraph(ContentWorkflowState)

# Add nodes
workflow.add_node("generate_article", generate_article_node)
workflow.add_node("summarize_article", summarize_article_node)

# Set entry point
workflow.set_entry_point("generate_article")

# Define edges
workflow.add_edge("generate_article", "summarize_article")
workflow.add_edge("summarize_article", END)

# Compile the graph
app = workflow.compile()

print("LangGraph workflow compiled!")

### Task 3.2: Execute the Workflow with Streaming Observation
Run the compiled `app` with a sample topic. Pay close attention to the console output to observe the streaming chunks being printed in real-time.

In [ ]:
sample_topic = "The Benefits of Renewable Energy Sources"

print(f"\n\n=========== Starting Workflow for Topic: '{sample_topic}' ===========")
initial_state = {
    "topic": sample_topic,
    "generated_article": "",
    "summary": ""
}

final_state = None
try:
    # Using .stream() on the compiled graph itself to get state updates
    # The streaming output from the LLM is handled within the nodes.
    for state_update in app.stream(initial_state):
        print(f"\n[LangGraph State Update]: {state_update}")
        final_state = state_update

    print("\n--- Workflow Execution Complete --- ")
    print("\nFinal Article:\n", final_state.get("generated_article", "N/A"))
    print("\nFinal Summary:\n", final_state.get("summary", "N/A"))
except Exception as e:
    print(f"An error occurred during workflow execution: {e}")

---

## Part 4: Analysis and Reflection
Provide a comprehensive summary of your findings and reflections based on this assignment.

* **Observation of Streaming**: Did you observe the content and summary being printed character by character or word by word? Describe your observation.
* **Implementation Details**: Explain how you implemented streaming within each node. What specific method or approach did you use (`.stream()`, `yield`, etc.) and why?
* **Benefits of Streaming**: Discuss the advantages of using streaming responses in LLM applications, especially in the context of user experience and perceived performance.
* **Challenges/Considerations**: What are some potential challenges or considerations when implementing streaming in a multi-node workflow (e.g., handling partial outputs, error handling, integrating with UI)?
* **Real-world Applications**: Provide examples of real-world applications where streaming LLM responses would be crucial or highly beneficial.

---

### Submission:
* Ensure all code cells have been executed and their outputs are visible.
* All analysis and reflections are clearly written in markdown cells.
* Save your Jupyter Notebook as `[YourName]_LangGraph_Streaming_Assignment.ipynb`.